In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

from keras.preprocessing.text import one_hot                              #for one_hot encoding labels(not used)
from keras.utils.np_utils import to_categorical                           # for one_hot encoding labels
from sklearn.model_selection import train_test_split
from keras.models import Sequential                                        #importing Sequential model
from keras.optimizers import Adam                                          #Importing Adam Optimizer

import os
import cv2                                                                 #Read Image using myCv
import glob  

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

from keras.losses import CategoricalCrossentropy                           #for loss function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt                                            #for plotting images and to display variations
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#Importing Required Data from Kaggle Kernels

In [ ]:
path_Train_Images = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train/"
path_test_Images =  "/kaggle/input/state-farm-distracted-driver-detection/imgs/test/"              #Path for importing train and test imgs

def prepare_Data(path): 
    imgsList = []
    labels = []
    for directory in sorted(glob.glob(os.path.join(path, '*')), key = lambda k: k.split("/")[-1]): #Exploring image directories
            for imgs in glob.glob(os.path.join(directory,'*.jpg')):                                #provided by kaggle kernel 
                img_cv = cv2.imread(imgs)                                                          #using glob path expansion 
                img_cv_r = cv2.resize(img_cv,(128,128))                                            #Ref Tutorial Points and kaggle
                imgsList.append(img_cv_r)                                                          #Creating List of resized images
                labels.append(int(directory.split("/")[-1].replace('c','')))
    
    X_Train, X_Test, Y_Train, Y_Test =  train_test_split(imgsList,labels, test_size = 0.2)         #Splitting on 8:2 ratio
    Y_Train = tf.keras.utils.to_categorical(Y_Train, num_classes=10)                               #one_hot labels with depth=10
    Y_Test = tf.keras.utils.to_categorical(Y_Test, num_classes=10)

    return np.array(X_Train), np.array(X_Test), Y_Train, Y_Test                                    #Returning Prepared Data



#List of Images for Train and Test                                                               
X_Train, X_Test, Y_Train, Y_Test = prepare_Data(path_Train_Images)

print("X_Train: {} ,Y_Train: {}".format(len(X_Train),len(Y_Train)))
print("X_Test: {},Y_Test: {}".format(len(X_Test),len(Y_Test)))                                     #Printing output values

In [ ]:
print(len(X_Train))
print(X_Train[100].shape)
img = X_Train[100]

RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Refr:"https://note.nkmk.me/en/python-opencv-bgr-rgb-cvtcolor/"
plt.imshow(RGB_img)                           #Converting BGR to RBG using cv2 and plotting the image of 100th trainind data
plt.show()
print("Class for  {} is :".format(Y_Train[100]))

In [ ]:
model = keras.models.Sequential()             #Model: Sequential 

model.add(keras.layers.InputLayer(            #Input Layer with 3 channels
    input_shape=(128, 128, 3)
))

model.add(
    keras.layers.Conv2D(
        filters=32,
        kernel_size=(5,5),                    #Conv_1 Layer  with 32 units
        strides = (1,1),
        padding='same',
        activation='relu',
        name='Conv_1'))

model.add(
    keras.layers.MaxPool2D(                   #Refr:"https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D"
        pool_size = (2,2),                    #Pooling Layer 1
        name = 'Pool_1'))                     #Image: 64*64*32

model.add(
    keras.layers.Conv2D(
        filters = 64,
        kernel_size = (5,5),
        strides = (1,1),
        padding = 'same',                     #Conv_2 layer with 64 units
        activation = 'relu',
        name = 'Conv_2'))

model.add(
    keras.layers.MaxPool2D(
        pool_size = (2,2),                    #Pooling Layer 2
        name = 'Pool_2'))                     #Image_size: 64*32*32(64 filters,image_size 32*32)

model.add(
    keras.layers.Conv2D(
        filters = 128,
        kernel_size = (5,5),
        strides = (1,1),
        padding = 'same',                     #Conv_3 Layer
        activation = 'relu',
        name = 'Conv_3'))

model.add(
    keras.layers.MaxPool2D(
        pool_size = (2,2),                    #Pooling Layer 3 
        name = 'Pool_3'))                     #Image_size:16*16*128

model.add(
    keras.layers.Conv2D(
        filters = 256,            
        kernel_size = (5,5),                  #Conv_3 Layer
        strides = (1,1),
        padding = 'same',
        activation = 'relu',
        name = 'Conv_4'))

model.add(
    keras.layers.MaxPool2D(
        pool_size = (2,2),
        name = 'Pool_4'))                     #Image_size: 8*8*256

model.add(keras.layers.Flatten())             #Flattening before entering Hidden Layer or # reshape(-1,16,16,256)

model.add(keras.layers.Dense(units=1024, activation='relu',name = 'hidden_1'))

model.add(keras.layers.Dropout(rate=0.2))     #Refr:"https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout"
                                              #Reduces Noise to prevent overfitting according to tensorflow

model.add(keras.layers.Dense(units=512, activation='relu',name = 'hidden_2'))

model.add(keras.layers.Dense(units=10,activation='softmax',name = 'hidden_3'))


In [ ]:
#Fitting the model using respective optimizers

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics = ['accuracy'])

In [ ]:
var = model.fit(x = X_Train, y= Y_Train, epochs = 10, batch_size = 500, verbose = 1,validation_split=0.2) #verbose=1 to see how epochcs to run

In [ ]:
test_loss_values, test_data_accuracy = model.evaluate(X_Test, Y_Test, verbose = 1)

In [ ]:
df = pd.DataFrame({'image':[],'label_0':[], 'label_1':[],'label_2':[], 'label_3':[], 'label_4':[],'label_5':[], 'label_6':[], 'label_7':[], 'label_8':[], 'label_9':[]})

def represent_predicted_(path_of_Images,df):
    
    for imgs in glob.glob(os.path.join(path_of_Images,'*.jpg')):
        
        img_cv = cv2.imread(imgs)
        img_cv_r = cv2.resize(img_cv,(128,128))
        
        img_cv_predict = np.reshape(img_cv_r,[1,128,128,3])
        arr_predict = model.predict(img_cv_predict,batch_size = 1)         #predicting for each and evry image and one_hot encoding
        #print(imgs.split('/')[-1])                                        # representation of predictions of labels
        df = df.append( 
            {
                'image':imgs.split('/')[-1],
                'label_0':round(arr_predict[0][0],2), 
                'label_1':round(arr_predict[0][1],2),
                'label_2':round(arr_predict[0][2],2),                      #values rounding off to 2 decimal points
                'label_3':round(arr_predict[0][3],2),                      #for better representation (avoiding in exponential form)
                'label_4':round(arr_predict[0][4],2),
                'label_5':round(arr_predict[0][5],2),
                'label_6':round(arr_predict[0][6],2),
                'label_7':round(arr_predict[0][7],2),
                'label_8':round(arr_predict[0][8],2),
                'label_9':round(arr_predict[0][9],2)
            },
            ignore_index=True
        )
    return df

In [ ]:
path_test_Images =  "/kaggle/input/state-farm-distracted-driver-detection/imgs/test/"
df = represent_predicted_(path_test_Images,df)    #Dataframe Containing Images and their predicitons

In [ ]:
print(df.shape)
df.head(50)